# Google Play Store Dataset – Data Overview & Quality Analysis

## Cleaning, Imputation, Outliers

In [227]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [228]:
# reading the dataset
df = pd.read_csv("C:\\Users\\anura\\Downloads\\googleplaystore_data.csv")
df

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10836,Sya9a Maroc - FR,FAMILY,4.5,38,53M,"5,000+",Free,0,Everyone,Education,"July 25, 2017",1.48,4.1 and up
10837,Fr. Mike Schmitz Audio Teachings,FAMILY,5.0,4,3.6M,100+,Free,0,Everyone,Education,"July 6, 2018",1.0,4.1 and up
10838,Parkinson Exercices FR,MEDICAL,NaN,3,9.5M,"1,000+",Free,0,Everyone,Medical,"January 20, 2017",1.0,2.2 and up
10839,The SCP Foundation DB fr nn5n,BOOKS_AND_REFERENCE,4.5,114,Varies with device,"1,000+",Free,0,Mature 17+,Books & Reference,"January 19, 2015",Varies with device,Varies with device


In [229]:
df.shape

(10841, 13)

In [230]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10841 entries, 0 to 10840
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   App             10841 non-null  object 
 1   Category        10841 non-null  object 
 2   Rating          9367 non-null   float64
 3   Reviews         10841 non-null  object 
 4   Size            10841 non-null  object 
 5   Installs        10841 non-null  object 
 6   Type            10840 non-null  object 
 7   Price           10841 non-null  object 
 8   Content Rating  10840 non-null  object 
 9   Genres          10841 non-null  object 
 10  Last Updated    10841 non-null  object 
 11  Current Ver     10833 non-null  object 
 12  Android Ver     10838 non-null  object 
dtypes: float64(1), object(12)
memory usage: 1.1+ MB


### Fixing Datatypes

##### Converting the datatype of 'Reviews' column from object to numeric type.

In [231]:
df['Reviews'] = pd.to_numeric(df['Reviews'],errors ='coerce')

##### Replacing the unnescessary symbols and converting the datatype of column 'Installs' to numeric type.

In [232]:
df['Installs']=(df['Installs'].str.replace('+','',regex=False).str.replace(',','',regex=False))
df['Installs']=pd.to_numeric(df['Installs'],errors='coerce')       

##### Replacing the unnescessary symbols and converting the datatype of column 'Price' to numeric type.

In [233]:
df['Price'] = df['Price'].str.replace('$','',regex=False)
df['Price'] = pd.to_numeric(df['Price'],errors='coerce')

##### Converting the datatype of 'Last_Updated' column from object to numeric type.

In [234]:
df['Last Updated'] = pd.to_datetime(df['Last Updated'],errors='coerce') 

##### The Size column was cleaned for consistency, non-measurable values were removed, and a new numeric feature 'Size_MB' was derived using regex-based unit parsing while preserving the original column.

In [235]:
df['Size'] = df['Size'].astype(str)
df['Size'] = df['Size'].str.strip().str.lower()

In [236]:
df.loc[df['Size'].str.contains('varies', na=False), 'Size'] = np.nan

In [237]:
def size_to_mb(size):
    if pd.isna(size):
        return np.nan
    
    if re.match(r'^\d+(\.\d+)?m$', size):
        return float(size.replace('m', ''))
    
    if re.match(r'^\d+(\.\d+)?k$', size):
        return float(size.replace('k', '')) / 1024
    
    return np.nan


In [238]:
df['Size_MB'] = df['Size'].apply(size_to_mb)

In [239]:
df[['Size', 'Size_MB']].head(15)
df['Size_MB'].isna().sum()

np.int64(1696)

##### I applied log1p transformation to highly skewed variables like installs and reviews to reduce skewness, stabilize variance, and improve interpretability in visualizations and statistical analysis.

In [240]:
df['Log_Reviews'] = np.log1p(df['Reviews'])
df['Log_Installs'] = np.log1p(df['Installs'])

### Standardize Categorical Variables

In [241]:
# Trim spaces and unify text format
cat_cols = df.select_dtypes(include='object').columns

for col in cat_cols:
    df[col] = df[col].str.strip().str.lower()

In [242]:
df['Type'] = df['Type'].replace({'free': 'Free', 'paid': 'Paid'})

### Remove Duplicate Records

In [243]:
df.duplicated().any()

np.True_

In [244]:
df.duplicated().sum()

np.int64(483)

In [245]:
# same app appears multiple times so to keep the most reviewed version first sort and then drop
df = df.sort_values('Reviews', ascending=False)
df = df.drop_duplicates(subset='App', keep='first')

### Handling Missing Values

In [246]:
df.isnull().sum()

App                  0
Category             0
Rating            1458
Reviews              1
Size              1226
Installs             1
Type                 1
Price                1
Content Rating       1
Genres               0
Last Updated         1
Current Ver          8
Android Ver          3
Size_MB           1227
Log_Reviews          1
Log_Installs         1
dtype: int64

##### Converted the missing values in the 'Rating' column with the median value

In [247]:
df['Rating'] = df['Rating'].fillna(df['Rating'].median())

##### Converted the missing values in the 'Type' column with the mode value

In [248]:
df['Type'] = df['Type'].fillna(df['Type'].mode()[0])

##### Converted the missing values in the 'Content Rating' column with the mode value

In [249]:
df['Content Rating'] = df['Content Rating'].fillna(df['Content Rating'].mode()[0])

##### The columns ('Category','Installs') are essential but the rows without them provide no business value

In [250]:
df = df.dropna(subset=['Category', 'Installs'])

### Outlier Detection

<!-- Outlier detection was performed on Rating and Reviews as they are continuous,
high-impact variables representing user feedback and engagement. These columns
exhibited skewness and extreme values during EDA, and outliers in them could
significantly affect analysis and model performance. Both IQR and Z-score
methods were used to ensure robustness under different distribution assumptions. -->

##### Finding the outliers in the column 'Rating' using the IQR method

In [251]:
Q1 = df['Rating'].quantile(0.25)
Q3 = df['Rating'].quantile(0.75)

In [252]:
IQR = Q3-Q1

In [253]:
lower = Q1 - 1.5 * IQR
upper = Q3 + 1.5 * IQR
lower,upper

(np.float64(3.25), np.float64(5.25))

In [254]:
outliers_iqr = df[(df['Rating']<lower) | (df['Rating']>upper)]
outliers_iqr.shape 

(490, 16)

##### Finding the outliers in the column 'Reviews' using the Z score method and created new column to store that z-values

In [255]:
from scipy.stats import zscore

df['Reviews_z'] = zscore(df['Reviews'].fillna(0))
outliers_z = df[df['Reviews_z'].abs() > 3]
outliers_z.shape

(68, 17)

In [256]:
df

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver,Size_MB,Log_Reviews,Log_Installs,Reviews_z
2544,facebook,social,4.1,78158306.0,NaN,1.000000e+09,Free,0.0,teen,social,2018-08-03,varies with device,varies with device,NaN,18.174247,20.723266,42.528864
381,whatsapp messenger,communication,4.4,69119316.0,NaN,1.000000e+09,Free,0.0,everyone,communication,2018-08-03,varies with device,varies with device,NaN,18.051345,20.723266,37.596732
2604,instagram,social,4.5,66577446.0,NaN,1.000000e+09,Free,0.0,teen,social,2018-07-31,varies with device,varies with device,NaN,18.013876,20.723266,36.209758
382,messenger – text and video chat for free,communication,4.0,56646578.0,NaN,1.000000e+09,Free,0.0,everyone,communication,2018-08-01,varies with device,varies with device,NaN,17.852342,20.723266,30.790973
1879,clash of clans,game,4.6,44893888.0,98m,1.000000e+08,Free,0.0,everyone 10+,strategy,2018-07-15,10.322.16,4.1 and up,98.0,17.619812,18.420681,24.378108
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7581,in to cm,tools,4.3,0.0,2.3m,1.000000e+01,Free,0.0,everyone,tools,2018-04-05,1.1,4.0.3 and up,2.3,0.000000,2.397895,-0.118276
9532,ek-yatri: travel where you belong,travel_and_local,4.3,0.0,29m,1.000000e+00,Free,0.0,everyone,travel & local,2018-08-01,1.0.0,4.4 and up,29.0,0.000000,0.693147,-0.118276
10822,plmgss fr,productivity,4.3,0.0,3.1m,1.000000e+01,Free,0.0,everyone,productivity,2017-12-01,1,4.4 and up,3.1,0.000000,2.397895,-0.118276
10835,fr forms,business,4.3,0.0,9.6m,1.000000e+01,Free,0.0,everyone,business,2016-09-29,1.1.5,4.0 and up,9.6,0.000000,2.397895,-0.118276


In [257]:
df.to_csv('googleplaystore_cleaned_interim.csv', index=False)

I performed systematic data cleaning by fixing data types, standardizing categorical variables, handling missing values using statistical reasoning, removing duplicates based on business logic, detecting outliers using IQR and Z-score, applying transformations for skewed data, and saving an interim cleaned dataset for EDA